In [1]:
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
    
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [2]:
import re
import os
import pandas as pd
import numpy as np

### Basic regular expression

In [3]:
m = re.finditer(r'.*?(spark).*?', "I'm searching for a spark in PySpark", re.I)
for match in m:
    print(match, match.start(), match.end())

<re.Match object; span=(0, 25), match="I'm searching for a spark"> 0 25
<re.Match object; span=(25, 36), match=' in PySpark'> 25 36


## Spark setup 
It can be setup as described below or through bashrc file.

In [4]:
os.environ["SPARK_HOME"] = "/opt/spark-2.4.5-bin-hadoop2.7"
print(os.environ.get('SPARK_HOME'))

/opt/spark-2.4.5-bin-hadoop2.7


## Download data
1) Download it if they do not exist locally.

In [5]:
path_to_data_dir = "./data/"
if not os.path.exists(path_to_data_dir):
    os.mkdir(path_to_data_dir)
    print("Created data directory")

if not os.path.exists("./data/NASA_access_log_Jul95.gz"):
    !wget ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz 
    !mv ./NASA_access_log_Jul95.gz ./data/

if not os.path.exists("./data/NASA_access_log_Aug95.gz"):
    !wget ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz 
    !mv ./NASA_access_log_Aug95.gz ./data/

if not os.path.exists("./data/clarknet_access_log_Aug28.gz"):
    !wget ftp://ita.ee.lbl.gov/traces/clarknet_access_log_Aug28.gz 
    !mv ./clarknet_access_log_Aug28.gz ./data/

if not os.path.exists("./data/clarknet_access_log_Sep4.gz"):
    !wget ftp://ita.ee.lbl.gov/traces/clarknet_access_log_Sep4.gz
    !mv ./clarknet_access_log_Sep4.gz ./data/

## Parse log files

Four gz files were downloaded and they have same format with same number of columns.

In [6]:
import glob
import os

raw_data_files = glob.glob('./data/*.gz')
print(raw_data_files)
base_df = spark.read.text(raw_data_files)
print(base_df.printSchema())
print(type(base_df))

['./data/clarknet_access_log_Sep4.gz', './data/NASA_access_log_Jul95.gz', './data/clarknet_access_log_Aug28.gz', './data/NASA_access_log_Aug95.gz']
root
 |-- value: string (nullable = true)

None
<class 'pyspark.sql.dataframe.DataFrame'>


## Data Wrangling

Converting data frame to Resilient Distributed Datasets (RDDs)

In [7]:
base_df.show(2, truncate=False)

base_df_rdd = base_df.rdd
print(type(base_df_rdd))
base_df_rdd.take(2)



+------------------------------------------------------------------------------------------------------------------+
|value                                                                                                             |
+------------------------------------------------------------------------------------------------------------------+
|204.249.225.59 - - [28/Aug/1995:00:00:34 -0400] "GET /pub/rmharris/catalogs/dawsocat/intro.html HTTP/1.0" 200 3542|
|access9.accsyst.com - - [28/Aug/1995:00:00:35 -0400] "GET /pub/robert/past99.gif HTTP/1.0" 200 4993               |
+------------------------------------------------------------------------------------------------------------------+
only showing top 2 rows

<class 'pyspark.rdd.RDD'>


[Row(value='204.249.225.59 - - [28/Aug/1995:00:00:34 -0400] "GET /pub/rmharris/catalogs/dawsocat/intro.html HTTP/1.0" 200 3542'),
 Row(value='access9.accsyst.com - - [28/Aug/1995:00:00:35 -0400] "GET /pub/robert/past99.gif HTTP/1.0" 200 4993')]

In [8]:
# !pip3 install findspark
# os.environ.get('SPARK_HOME')
import findspark
findspark.init()

### Clean data

There are multiple columns that are listed below. They have their unique patterns.  

1. Host names
2. Access timestamps
3. HTTP Request method, endpoint, and protocol
4. HTTP Status codes
5. HTTP Response content size

In [9]:
pa_host = r'(^\S+\.[\S+\.]+\S+)\s' # host name - ['204.249.225.59', 'access9.accsyst.com']
pa_time = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]' # date and time - 01/Jul/1995:00:00:01 -0400
pa_protocol = r'\"(\S+)\s(\S+)\s*(\S*)\"' # ('GET', '/history/apollo/', 'HTTP/1.0')
pa_status = r'\s(\d{3})\s' # status code
pa_content_size = r'\s(\d+)$' # Content size

In [10]:
from pyspark.sql.functions import regexp_extract

df_parse_log = base_df.select(regexp_extract('value', pa_host, 1).alias('Host'),
                         regexp_extract('value', pa_time, 1).alias('Timestamp'),
                         regexp_extract('value', pa_protocol, 1).alias('Method'),
                         regexp_extract('value', pa_protocol, 2).alias('Endpoint'),
                         regexp_extract('value', pa_protocol, 3).alias('Protocol'),
                         regexp_extract('value', pa_status, 1).cast('integer').alias('Status'),
                         regexp_extract('value', pa_content_size, 1).cast('integer').alias('Content_size'))
df_parse_log.show(2, truncate=True)
print((df_parse_log.count, len(df_parse_log.columns)))

+-------------------+--------------------+------+--------------------+--------+------+------------+
|               Host|           Timestamp|Method|            Endpoint|Protocol|Status|Content_size|
+-------------------+--------------------+------+--------------------+--------+------+------------+
|     204.249.225.59|28/Aug/1995:00:00...|   GET|/pub/rmharris/cat...|HTTP/1.0|   200|        3542|
|access9.accsyst.com|28/Aug/1995:00:00...|   GET|/pub/robert/past9...|HTTP/1.0|   200|        4993|
+-------------------+--------------------+------+--------------------+--------+------+------------+
only showing top 2 rows

(<bound method DataFrame.count of DataFrame[Host: string, Timestamp: string, Method: string, Endpoint: string, Protocol: string, Status: int, Content_size: int]>, 7)


## Finding missing values

In [11]:
df_missing_data = df_parse_log.filter(df_parse_log['Host'].isNull()| 
                                    df_parse_log['Timestamp'].isNull() | 
                                    df_parse_log['Method'].isNull() |
                                    df_parse_log['Endpoint'].isNull() |
                                    df_parse_log['Protocol'].isNull() |
                                    df_parse_log['Status'].isNull() |
                                    df_parse_log['Content_size'].isNull())

In [12]:
df_missing_data.show(2)

+------------------+--------------------+------+--------------------+--------+------+------------+
|              Host|           Timestamp|Method|            Endpoint|Protocol|Status|Content_size|
+------------------+--------------------+------+--------------------+--------+------+------------+
|ppp19.glas.apc.org|28/Aug/1995:00:00...|   GET|          /atomicbk/|HTTP/1.0|   304|        null|
|ppp19.glas.apc.org|28/Aug/1995:00:00...|   GET|/atomicbk/contest...|HTTP/1.0|   304|        null|
+------------------+--------------------+------+--------------------+--------+------+------------+
only showing top 2 rows



What is distribution of missing values per column?

-> Build up a list of column expressions, one per column.

-> Run the aggregation. The *exprs converts the list of expressions into

-> variable function arguments.

In [13]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum

def count_null(col_name):
    return spark_sum(col(col_name).isNull().cast('integer')).alias(col_name)

exprs = [count_null(col_name) for col_name in df_parse_log.columns]
df_parse_log.agg(*exprs).show()

+----+---------+------+--------+--------+------+------------+
|Host|Timestamp|Method|Endpoint|Protocol|Status|Content_size|
+----+---------+------+--------+--------+------+------------+
|   0|        0|     0|       0|       0|   101|      308120|
+----+---------+------+--------+--------+------+------------+



In [14]:
null_status_df = base_df.filter(~base_df['value'].rlike(r'\s(\d{3})\s'))
null_status_df.show(1)

+--------------------+
|               value|
+--------------------+
|rodan29.pixi.com ...|
+--------------------+
only showing top 1 row



In [15]:
bad_status_df = null_status_df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                                      regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                                      regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                                      regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                                      regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                                      regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                                      regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))
bad_status_df.show(truncate=False)

NameError: name 'host_pattern' is not defined

In [ ]:
df_parse_log = df_parse_log[df_parse_log['Status'].isNotNull()]
df_parse_log.show(2)

In [ ]:
exprs = [count_null(col_name) for col_name in df_parse_log.columns]
df_parse_log.agg(*exprs).show(2)

## Handling nulls in HTTP content size

In [ ]:
null_content_size_df = base_df.filter(~base_df['value'].rlike(r'\s\d+$'))
null_content_size_df.show(1, truncate=True)

In [ ]:
null_content_size_df.take(2)

In [ ]:
df_parse_log = df_parse_log.na.fill({'content_size': 0})

In [ ]:
exprs = [count_null(col_name) for col_name in logs_df.columns]
logs_df.agg(*exprs).show()

## Handling Temporal Fields (Timestamp)

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType
from datetime import datetime

month_str_ints = {'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4 , 'May': 5, 'Jun':6, 
                  'Jul': 7, 'Aug': 8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12 }

def parse_clf_time(text):
    """
      Keep date and time hours, minutes and seconds
    """
    
    cleaned_time = "{0:04d}/{1:02d}/{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
                      int(text[7:11]),
                      month_str_ints[text[3:6]],
                      int(text[0:2]),
                      int(text[12:14]),
                      int(text[15:17]),
                      int(text[18:20]))
    return cleaned_time
  

In [17]:
from pyspark.sql.functions import unix_timestamp
udf_parse_time = udf(parse_clf_time)
a = df_parse_log.select('*', unix_timestamp(udf_parse_time(df_parse_log['Timestamp']), "yyyy/MM/dd HH:mm:ss")
                                .cast(TimestampType())
                                .alias('Time')).drop('Timestamp')

In [18]:
a.show()

+--------------------+------+--------------------+--------+------+------------+-------------------+
|                Host|Method|            Endpoint|Protocol|Status|Content_size|               Time|
+--------------------+------+--------------------+--------+------+------------+-------------------+
|      204.249.225.59|   GET|/pub/rmharris/cat...|HTTP/1.0|   200|        3542|1995-08-28 00:00:34|
| access9.accsyst.com|   GET|/pub/robert/past9...|HTTP/1.0|   200|        4993|1995-08-28 00:00:35|
| access9.accsyst.com|   GET|/pub/robert/curr9...|HTTP/1.0|   200|        5836|1995-08-28 00:00:35|
|       world.std.com|   GET|/pub/atomicbk/cat...|HTTP/1.0|   200|       18338|1995-08-28 00:00:36|
|    cssu24.cs.ust.hk|   GET|/pub/job/vk/view1...|HTTP/1.0|   200|        5944|1995-08-28 00:00:36|
|     er6.rutgers.edu|   GET|/pub/rjgula/netwo...|HTTP/1.0|   200|        2017|1995-08-28 00:00:37|
|cyclom1-1-6.inter...|   GET|/pub/k2/jeep/jxj.htm|HTTP/1.0|   200|        3254|1995-08-28 00:00:37|


In [ ]:
logs_df.show()

In [ ]:
a.printSchema()

In [ ]:
a.show()

# Explortory Data Analysis (EDA)

## HTTP Status
First, unique HTTP status is determined and then find their distribution.



In [ ]:
df_status_freq = (logs_df
                     .groupBy('status')
                     .count()
                     .sort('status')
                     .cache())

In [ ]:
df_pd_status_freq = (df_status_freq
                         .toPandas()
                         .sort_values(by=['count'], ascending=False))

df_pd_status_freq['percent'] = df_pd_status_freq['count']/sum(df_pd_status_freq['count'])
df_pd_status_freq['log(count)'] = np.log(df_pd_status_freq['count'])
df_pd_status_freq


In [ ]:
df_pd_status_freq.head()

## Frequent host

Identify the top ten host and also compute their proportion.

In [ ]:
logs_df.show(2)

In [ ]:
df_host_freq = (logs_df
                     .groupBy('host')
                     .count()
                     .cache())
df_pd_host_freq = (df_host_freq
                        .toPandas()
                        .sort_values(by=['count'], ascending=False))
df_pd_host_freq["Percentage"] = df_pd_host_freq["count"] / sum(df_pd_host_freq["count"])
df_pd_host_freq["Log(count)"] = np.log(df_pd_host_freq["count"])

In [ ]:
df_pd_host_freq.head(3)

In [ ]:
mean    = np.mean(df_pd_host_freq['count'])
std     = np.std(df_pd_host_freq['count'])
df_pd_host_freq['zscore'] = (df_pd_host_freq['count'] - mean)/std
df_pd_host_freq.head()

## What method is most frequently used?

In [ ]:
df_method = (logs_df
                     .groupBy('method')
                     .count()
                     .cache())
df_pd_method = (df_method
                        .toPandas()
                        .sort_values(by=['count'], ascending=False))
df_pd_method["Percentage"] = 100.00 * (df_pd_method["count"] / sum(df_pd_method["count"]))
df_pd_method["Log(count)"] = np.log(df_pd_method["count"])

In [ ]:
df_pd_method.head()

Most popular method is GET followed by HEAD. However, 99% of times GET method is used. Superisingly, POST method is almost never used.  

In [ ]:
df_protocol = (logs_df
                     .groupBy('protocol')
                     .count()
                     .cache())
df_protocol = (df_protocol
                        .toPandas()
                        .sort_values(by=['count'], ascending=False))
df_protocol["Percentage"] = 100.00 * (df_protocol["count"] / sum(df_protocol["count"]))
df_protocol["Log(count)"] = np.log(df_protocol["count"])

In [ ]:
df_protocol.head()